## Implementação da solução

### Carregando os dados...

In [4]:
"""
Imports
"""
import codecs
import pandas as pd

In [2]:
def read_data_to_df():
    """ Read data into dataframe
    """
    df = pd.DataFrame()
    filenames = ['data/rt-polarity.neg', 'data/rt-polarity.pos']
    for filename in filenames:
        with codecs.open(filename, 'r', 'utf-8') as f:
            lines = f.readlines()
        # remove breaklines if they exist
        lines = [u' '.join(line.split()).strip() for line in lines]
        df_aux = pd.DataFrame()
        df_aux['review'] = lines
        positive_label = u'.pos' in filename
        df_aux['label'] = int(positive_label)
        df = pd.concat([df, df_aux])
    return df

In [6]:
df = read_data_to_df()
df.head()

,review,label
0,"simplistic , silly and tedious .",0
1,"it's so laddish and juvenile , only teenage bo...",0
2,exploitative and largely devoid of the depth o...,0
3,[garbus] discards the potential for pathologic...,0
4,a visually flashy but narratively opaque and e...,0
